<a href="https://colab.research.google.com/github/adamggibbs/marine-carbonate-system-ml-prediction/blob/master/data_quality_control_ship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SET UP ENVIRONMENT 

# mount google drive for data storage and access
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# IMPORTS

import os

import pandas as pd
import numpy as np


In [ ]:
# STORE DIRECTORY WHERE DATA IS STORED

# this is an absolute path 
# and be sure to include trailing '/'
data_dir = '/content/drive/MyDrive/Adam Gibbs/data/'
ship_dir = data_dir + 'ship/'
raw_ship_dir = ship_dir + 'raw/'

In [ ]:
# READ DATA INTO A PANDAS DATAFRAME TO DISPLAY

print('Files in ship data folder:')
display(os.listdir(raw_ship_dir))

dfs_ship= []
for file in os.listdir(raw_ship_dir):
  df = pd.read_csv(raw_ship_dir + file)
  dfs_ship.append(df.dropna().reset_index(drop=True))

print()
print("There are {0} dataframes from {0} shipboard data files".format(len(dfs_ship)))
print("Adjust the index below to toggle display.")
print()

display(dfs_ship[1])
display(dfs_ship[2])

Files in ship data folder:


['mltrain_ship052021.csv', 'mltrain_ship052019.csv', 'mltrain_ship072019.csv']


There are 3 dataframes from 3 shipboard data files
Adjust the index below to toggle display.



,EXPOCODE,SECT_ID,STNNBR,CASTNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,LONGITUDE,CTDPRS,CTDTMP,CTDSAL,CTDSAL_FLAG_W,SALNTY,SALNTY_FLAG_W,CTDOXY,CTDOXY_FLAG_W,OXYGEN,OXYGEN_FLAG_W,TCARBN,TCARBN_FLAG_W,ALKALI,ALKALI_FLAG_W,PH_TOT,PH_TOT_FLAG_W,PH_TMP,CHL_GFF,PHAEO_GFF,CHLPHAEO_FLAG_W,PHSPHT,PHSPHT_FLAG_W,SILCAT,SILCAT_FLAG_W,NITRAT,NITRAT_FLAG_W,NITRIT,NITRIT_FLAG_W,AMMONIUM,AMMONIUM_FLAG_W
0,31WF20190511,C1,1,1,1,1,20190511,627,36.7980,-121.8462,202.6,8.6075,33.9971,2,-999.0,9,188.6,4,94.2,2,2226.029657,2,2250.03,3,7.5056,2,20,0.134446,0.361090,2,2.257,2,39.993,2,27.910,2,0.432,2,0.222,2
1,31WF20190511,C1,1,1,2,1,20190511,627,36.7980,-121.8462,202.8,8.5810,34.0010,2,-999.0,9,188.1,4,93.3,2,2228.085668,2,2232.86,4,7.5022,2,20,0.106766,0.388770,2,2.161,2,39.197,2,27.544,2,0.460,2,0.227,2
2,31WF20190511,C1,1,1,3,1,20190511,629,36.7980,-121.8462,152.1,9.1717,33.9202,2,-999.0,9,173.6,4,112.5,2,2209.100812,2,2253.21,3,7.5416,2,20,0.111709,0.395958,2,1.960,2,26.017,4,20.104,4,0.330,2,0.412,2
3,31WF20190511,C1,1,1,4,1,20190511,630,36.7980,-121.8462,101.3,9.5935,33.8561,2,-999.0,9,154.5,4,129.6,2,2192.969821,2,2255.62,3,7.5770,2,20,0.109731,0.328878,2,1.735,2,29.691,2,24.523,2,0.398,2,0.440,2
4,31WF20190511,C1,1,1,5,1,20190511,632,36.7980,-121.8462,80.6,10.2153,33.7516,2,-999.0,9,140.8,4,159.8,2,2165.331307,6,2247.79,3,7.6346,6,20,0.108743,0.282273,2,1.327,4,19.516,2,18.488,2,0.340,2,0.730,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,31WF20190511,MW1,20,1,8,1,20190516,1723,36.6997,-122.0507,301.6,7.4797,34.1459,2,-999.0,9,50.2,2,-999.0,9,-999.000000,9,-999.00,9,-999.0000,9,20,0.021749,0.107968,2,2.387,2,45.264,2,34.196,2,0.083,2,-999.000,9
380,31WF20190511,MW1,20,1,9,1,20190516,1725,36.6997,-122.0507,202.2,8.1053,34.0509,2,-999.0,9,84.2,2,-999.0,9,-999.000000,9,-999.00,9,-999.0000,9,20,0.000000,2.616717,2,2.242,2,32.943,2,30.973,2,0.125,2,-999.000,9
381,31WF20190511,MW1,20,1,10,1,20190516,1728,36.6997,-122.0507,102.2,9.5789,33.8319,2,-999.0,9,126.6,2,-999.0,9,-999.000000,9,-999.00,9,-999.0000,9,20,0.037566,0.139744,2,1.746,2,15.626,2,24.146,2,0.095,2,-999.000,9
382,31WF20190511,MW1,20,1,11,1,20190516,1729,36.6997,-122.0507,51.5,10.8730,33.5166,2,-999.0,9,190.7,2,-999.0,9,-999.000000,9,-999.00,9,-999.0000,9,20,0.268891,0.226644,2,1.174,2,9.321,2,12.229,2,0.230,2,-999.000,9


,EXPOCODE,SECT_ID,STNNBR,CASTNO,BTLNBR,BTLNBR_FLAG_W,DATE,TIME,LATITUDE,LONGITUDE,CTDPRS,CTDTMP,CTDSAL,CTDSAL_FLAG_W,SALNTY,SALNTY_FLAG_W,CTDOXY,CTDOXY_FLAG_W,CTDPHIN_TOT,CTDPHIN_TOT_FLAG_W,OXYGEN,OXYGEN_FLAG_W,SILCAT,SILCAT_FLAG_W,NITRAT,NITRAT_FLAG_W,NITRIT,NITRIT_FLAG_W,PHSPHT,PHSPHT_FLAG_W,AMMONIA,AMMONIA_FLAG_W,TCARBN,TCARBN_FLAG_W,ALKALI,ALKALI_FLAG_W,PH_TOT,PH_TOT_FLAG_W,PHTMP,CHL_GFF,PHAEO_GFF,CHLPHAEO_FLAG_W
0,C3PO,C1,1,1,1,1,20190723,2311,36.7982,-121.8462,202.3,8.5788,34.0377,2,34.0378,2,76.3,2,7.5775,2,101.9,4,43.182,2,31.881,2,0.118,2,2.660,4,0.049,2,2236.39,2,2259.50,2,7.478651,2,20,0.11,0.27,2
1,C3PO,C1,1,1,2,1,20190723,2311,36.7982,-121.8462,202.1,8.5774,34.0382,2,-999.0000,9,76.3,2,7.5797,2,78.4,2,40.007,2,29.474,2,0.145,2,2.837,4,-999.000,9,2236.65,2,-999.00,9,7.476772,2,20,0.13,0.31,2
2,C3PO,C1,1,1,3,1,20190723,2313,36.7982,-121.8462,151.2,8.8875,33.9995,2,-999.0000,9,84.8,2,7.5987,2,86.2,2,30.830,2,24.863,4,0.076,2,2.097,2,0.111,2,2226.93,2,2249.60,2,7.492670,2,20,0.11,0.33,2
3,C3PO,C1,1,1,4,1,20190723,2314,36.7982,-121.8462,101.3,9.4388,33.9235,2,-999.0000,9,100.2,2,7.6253,2,100.8,2,32.856,2,28.613,2,0.124,2,1.991,2,0.026,2,2208.39,2,-999.00,9,7.522627,2,20,0.30,0.48,2
4,C3PO,C1,1,1,5,1,20190723,2316,36.7982,-121.8462,80.6,9.6299,33.8970,2,33.9025,2,107.3,2,7.6357,2,107.2,2,30.828,2,27.744,2,0.154,2,2.036,2,0.037,2,2202.01,2,2239.36,2,7.535321,2,20,0.41,0.74,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,C3PO,73_55,20,1,8,1,20190729,1122,35.4770,-121.6088,51.5,11.4272,33.5325,2,-999.0000,9,192.9,4,7.9007,2,112.7,2,11.288,2,14.023,2,0.363,2,1.384,2,-999.000,9,2111.97,2,-999.00,9,7.577144,2,20,-999.00,-999.00,9
390,C3PO,73_55,20,1,9,1,20190729,1123,35.4770,-121.6088,25.3,13.4387,33.4912,2,-999.0000,9,234.7,4,8.0312,2,189.7,2,4.048,2,4.032,2,0.286,2,0.890,2,-999.000,9,2063.08,2,2236.52,2,7.746035,2,20,-999.00,-999.00,9
391,C3PO,73_55,20,1,10,1,20190729,1124,35.4770,-121.6088,10.7,15.8185,33.4956,2,-999.0000,9,260.9,4,8.1212,2,237.7,2,1.609,2,0.725,2,0.151,2,0.585,2,-999.000,9,2020.79,2,-999.00,9,7.890863,2,20,-999.00,-999.00,9
392,C3PO,73_55,20,1,11,1,20190729,1125,35.4770,-121.6088,5.3,15.8475,33.4995,2,-999.0000,9,262.2,2,8.1420,2,259.6,2,1.259,2,0.214,2,0.196,2,0.404,2,-999.000,9,2010.13,2,2238.70,2,7.990843,2,20,-999.00,-999.00,9


In [ ]:
def check_qfs(file, save_csv=True):
  # read in csv
  dataframe = pd.read_csv(file)

  df = dataframe[['PH_TOT', 'PH_TOT_FLAG_W', 'CTDSAL', 'CTDSAL_FLAG_W', 
                  'OXYGEN', 'OXYGEN_FLAG_W']].to_numpy()

  for ind in range(0, len(df)):
    # check pH QFs
    pH = df[ind][0]
    qf = df[ind][1]
    if pH < 7.3 or pH > 8.5:
      df[ind][1] = 8
    elif ind > 1 and ind + 2 < len(df):
      test = pH - np.median((df[ind-2][0], df[ind-1][0], 
                              pH, 
                              df[ind+1][0], df[ind+2][0]))
      if abs(test) > 0.04:
        qf = 8

    # check inputs QFs
    if float(df[ind][2]) == -999:
      df[ind][3] = 9
    if float(df[ind][4]) == -999:
      df[ind][5] = 9

  # update pH QFs
  dataframe['PH_INSITU_QF'] = df[:,1]
  # update inputs QFs
  dataframe['CTDSAL_FLAG_W'] = df[:,3]
  dataframe['OXYGEN_FLAG_W'] = df[:,5]

  if save_csv:
    dataframe.to_csv(ship_dir + file[-22:], header=True, index=False)

  return dataframe

In [ ]:
  # loop through all data files and add them to input array
  print('Parsing the following files:')
  for file in os.listdir(raw_ship_dir):
    print(raw_ship_dir + file)
    check_qfs(raw_ship_dir + file)
  print('Complete.')

Parsing the following files:
/content/drive/MyDrive/Adam Gibbs/data/ship/raw/mltrain_ship052021.csv
/content/drive/MyDrive/Adam Gibbs/data/ship/raw/mltrain_ship052019.csv
/content/drive/MyDrive/Adam Gibbs/data/ship/raw/mltrain_ship072019.csv
Complete.
